#### Geopandas example 
Uses the geopandas library to perform a spatial join of climate data and counties to assign FIPS codes to each 1/8th degree record.

Based on example in https://gis.stackexchange.com/questions/175228/geopandas-spatial-join-extremely-slow

In [1]:
import sys, os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from geopandas.tools import sjoin
from shapely.geometry import Point, mapping,shape

In [2]:
#Input files
dataDir = "..\\..\\Data"
countyFN = dataDir + os.sep + "cb_2016_us_county_5m.shp"
hydroFN = 'df2000.csv'
outFC = dataDir + os.sep + "HydroFIPS.csv"

In [13]:
#Create a dataframe from the hydro data
df=pd.read_csv(hydroFN,index_col=None,nrows=2000)

In [14]:
#Add geometry to the hydro data based on its Lat/Long values
##First create a column of shapely point objects from the long/lat values 
df['geometry'] = df.apply(lambda z: Point(z.LON, z.LAT), axis=1)

In [15]:
#Create a geopandas dataframe from the dataframe created above
PointsGeodataframe = gpd.GeoDataFrame(df)

In [16]:
#Create a geopandas dataframe from the countyies file
PolygonsGeodataframe = gpd.GeoDataFrame.from_file(countyFN)

In [17]:
#Drop unneeded columns
PolygonsGeodataframe.drop(['AFFGEOID','COUNTYFP','COUNTYNS','LSAD'],axis=1,inplace=True)

In [18]:
#Set the coordinate system of the points equal to the polygons
PointsGeodataframe.crs = PolygonsGeodataframe.crs

In [19]:
#Execute the spatial join
merged=sjoin(PointsGeodataframe, PolygonsGeodataframe, how='left',op='within')

In [25]:
merged['AREA'] = merged['ALAND'] + merged['AWATER']

KeyError: 'ALAND'

In [24]:
#Fix columns
merged.drop(['geometry','index_right','ALAND','AWATER'],axis=1,inplace=True)

In [26]:
#Drop missing data
merged.dropna(inplace=True)

In [27]:
#Write to a file
merged.to_csv(outFC,index=False,encoding='utf8')#,na_rep=-9999)